In [1]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import os
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

# Generate Train DFs

In [2]:
MY_PATH = 'ml100-03-final/image_data/'
TRAIN_PATH = 'train/'
TEST_PATH = 'test/'

In [3]:
daisy = os.listdir(MY_PATH+TRAIN_PATH+'/daisy')
dandelion = os.listdir(MY_PATH+TRAIN_PATH+'/dandelion')
rose = os.listdir(MY_PATH+TRAIN_PATH+'/rose')
sunflower = os.listdir(MY_PATH+TRAIN_PATH+'/sunflower')
tulip = os.listdir(MY_PATH+TRAIN_PATH+'/tulip')

In [4]:
# raw function def.
def raw(data,label):
    df=pd.DataFrame(data,columns=['dirname'])
    df['label']=[label]*len(data)
    return df

# dfs for 5 types of flowers
daisy_df=raw(daisy,[1,0,0,0,0])
dandelion_df=raw(dandelion,[0,1,0,0,0])
rose_df=raw(rose,[0,0,1,0,0])
sunflower_df=raw(sunflower,[0,0,0,1,0])
tulip_df=raw(tulip,[0,0,0,0,1])

daisy_df['folder']='daisy'
dandelion_df['folder']='dandelion'
rose_df['folder']='rose'
sunflower_df['folder']='sunflower'
tulip_df['folder']='tulip'

In [5]:
#每個資料集取 390 做train 剩下做各取98 做test

In [6]:
def reduce_sample(df):
    df=df[:488]
    return df

daisy_df=reduce_sample(daisy_df)
dandelion_df=reduce_sample(dandelion_df)
rose_df=reduce_sample(rose_df)
sunflower_df=reduce_sample(sunflower_df)
tulip_df=reduce_sample(tulip_df)

In [7]:
train_data = []
test_data = []

# funciotn that make train data
def load_data(IMAGE_SIZE=256):
    print("Loading images...")
    for data_df in [daisy_df,dandelion_df,rose_df,sunflower_df,tulip_df]:
        X_train, X_test, y_train, y_test = train_test_split(range(0,488),range(0,488),
                                                     test_size=0.2, random_state=1, shuffle = True)
        for i in X_train:
            #print("Loading {0}".format(dirname))
            dirnames=data_df.iloc[i]["dirname"]
            folder_path = data_df.iloc[i]['folder']+'/'
            label = data_df.iloc[i]['label']
            img = Image.open(MY_PATH + TRAIN_PATH + folder_path + dirnames)
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
            train_data.append([np.array(img),label])
            
        for i in X_test:
            #print("Loading {0}".format(dirname))
            dirnames=data_df.iloc[i]["dirname"]
            folder_path = data_df.iloc[i]['folder']+'/'
            label = data_df.iloc[i]['label']
            img = Image.open(MY_PATH + TRAIN_PATH + folder_path + dirnames)
            img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
            test_data.append([np.array(img),label])
    
    
    print("done")
    return train_data ,test_data

train_data, test_data = load_data()


Loading images...
done


In [8]:
len(train_data)

1950

In [9]:
len(test_data)

490

In [10]:
def create_dnn_model(IMAGE_SIZE=256):
    model = Sequential()
    #卷積網
    model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Flatten())
    
    #神經網
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation = 'softmax'))
    print('creating model')
    return model


In [11]:
model = create_dnn_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

creating model


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 127, 127, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 62, 62, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       7

In [13]:
training_images = np.array([i[0] for i in train_data])
training_labels = np.array([i[1] for i in train_data])
testing_images = np.array([i[0] for i in test_data])
testing_labels = np.array([i[1] for i in test_data])

In [14]:
# 影像 generator
def generator(batch_size=64):
    while True:
        for indexs in range(0, len(training_images), batch_size):
            images = training_images[indexs: indexs+batch_size]
            labels = training_labels [indexs: indexs+batch_size]
            yield images, labels
            
img_gen = generator()

In [15]:
checkpoint_dir = "training_1"
# 若目錄不存在，則新建
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                              save_weights_only=True,  # save_best_only=True
                                              verbose=1,
                                             monitor='val_loss')

In [16]:
model.fit_generator(img_gen, 
                    steps_per_epoch=(len(training_images) / 64),
                    epochs=150,
                    verbose=1,
                    validation_data=(testing_images, testing_labels),
                    callbacks=cp_callback)



Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/150
31/30 [==============================] - ETA: -2s - loss: 3.4553 - accuracy: 0.4354
Epoch 00001: saving model to training_1/cp.ckpt
31/30 [==============================] - 175s 6s/step - loss: 3.4553 - accuracy: 0.4354 - val_loss: 116.1879 - val_accuracy: 0.2000
Epoch 2/150
31/30 [==============================] - ETA: -2s - loss: 5.9544 - accuracy: 0.3195
Epoch 00002: saving model to training_1/cp.ckpt
31/30 [==============================] - 157s 5s/step - loss: 5.9544 - accuracy: 0.3195 - val_loss: 140.4200 - val_accuracy: 0.2082
Epoch 3/150
31/30 [==============================] - ETA: -2s - loss: 6.8644 - accuracy: 0.2697
Epoch 00003: saving model to training_1/cp.ckpt
31/30 [==============================] - 152s 5s/step - loss: 6.8644 - accuracy: 0.2697 - val_loss: 84.8638 - val_accuracy: 0.1918
Epoch 4/150
31/30 [==============================] - ETA: -2s - loss: 6.1105 - accuracy: 0.1944
E

Epoch 30/150
31/30 [==============================] - ETA: -2s - loss: 1.0732 - accuracy: 0.5277
Epoch 00030: saving model to training_1/cp.ckpt
31/30 [==============================] - 155s 5s/step - loss: 1.0732 - accuracy: 0.5277 - val_loss: 1.8721 - val_accuracy: 0.2837
Epoch 31/150
31/30 [==============================] - ETA: -2s - loss: 1.0133 - accuracy: 0.5769
Epoch 00031: saving model to training_1/cp.ckpt
31/30 [==============================] - 157s 5s/step - loss: 1.0133 - accuracy: 0.5769 - val_loss: 1.8985 - val_accuracy: 0.2898
Epoch 32/150
31/30 [==============================] - ETA: -2s - loss: 0.9836 - accuracy: 0.5882
Epoch 00032: saving model to training_1/cp.ckpt
31/30 [==============================] - 149s 5s/step - loss: 0.9836 - accuracy: 0.5882 - val_loss: 1.9514 - val_accuracy: 0.3061
Epoch 33/150
31/30 [==============================] - ETA: -2s - loss: 0.9570 - accuracy: 0.5764
Epoch 00033: saving model to training_1/cp.ckpt
31/30 [=======================

Epoch 60/150
31/30 [==============================] - ETA: -3s - loss: 0.3404 - accuracy: 0.8672
Epoch 00060: saving model to training_1/cp.ckpt
31/30 [==============================] - 204s 7s/step - loss: 0.3404 - accuracy: 0.8672 - val_loss: 2.8216 - val_accuracy: 0.3612
Epoch 61/150
31/30 [==============================] - ETA: -3s - loss: 0.3228 - accuracy: 0.8841
Epoch 00061: saving model to training_1/cp.ckpt
31/30 [==============================] - 205s 7s/step - loss: 0.3228 - accuracy: 0.8841 - val_loss: 2.8373 - val_accuracy: 0.3408
Epoch 62/150
31/30 [==============================] - ETA: -3s - loss: 0.3244 - accuracy: 0.8779
Epoch 00062: saving model to training_1/cp.ckpt
31/30 [==============================] - 192s 6s/step - loss: 0.3244 - accuracy: 0.8779 - val_loss: 2.8870 - val_accuracy: 0.3286
Epoch 63/150
31/30 [==============================] - ETA: -3s - loss: 0.2910 - accuracy: 0.8918
Epoch 00063: saving model to training_1/cp.ckpt
31/30 [=======================

Epoch 90/150
31/30 [==============================] - ETA: -2s - loss: 0.1192 - accuracy: 0.9574
Epoch 00090: saving model to training_1/cp.ckpt
31/30 [==============================] - 143s 5s/step - loss: 0.1192 - accuracy: 0.9574 - val_loss: 3.0930 - val_accuracy: 0.4265
Epoch 91/150
31/30 [==============================] - ETA: -43s - loss: 0.1150 - accuracy: 0.9600
Epoch 00091: saving model to training_1/cp.ckpt
31/30 [==============================] - 2556s 82s/step - loss: 0.1150 - accuracy: 0.9600 - val_loss: 3.3970 - val_accuracy: 0.4388
Epoch 92/150
31/30 [==============================] - ETA: -157s - loss: 0.1173 - accuracy: 0.9631
Epoch 00092: saving model to training_1/cp.ckpt
31/30 [==============================] - 9193s 297s/step - loss: 0.1173 - accuracy: 0.9631 - val_loss: 4.0715 - val_accuracy: 0.3796
Epoch 93/150
31/30 [==============================] - ETA: -2s - loss: 0.1043 - accuracy: 0.9579
Epoch 00093: saving model to training_1/cp.ckpt
31/30 [===============

Epoch 120/150
31/30 [==============================] - ETA: -2s - loss: 0.0715 - accuracy: 0.9744
Epoch 00120: saving model to training_1/cp.ckpt
31/30 [==============================] - 171s 6s/step - loss: 0.0715 - accuracy: 0.9744 - val_loss: 3.6928 - val_accuracy: 0.4122
Epoch 121/150
31/30 [==============================] - ETA: -2s - loss: 0.0841 - accuracy: 0.9744
Epoch 00121: saving model to training_1/cp.ckpt
31/30 [==============================] - 144s 5s/step - loss: 0.0841 - accuracy: 0.9744 - val_loss: 4.0945 - val_accuracy: 0.3816
Epoch 122/150
31/30 [==============================] - ETA: -3s - loss: 0.0953 - accuracy: 0.9682
Epoch 00122: saving model to training_1/cp.ckpt
31/30 [==============================] - 194s 6s/step - loss: 0.0953 - accuracy: 0.9682 - val_loss: 3.7699 - val_accuracy: 0.3816
Epoch 123/150
31/30 [==============================] - ETA: -3s - loss: 0.1531 - accuracy: 0.9631
Epoch 00123: saving model to training_1/cp.ckpt
31/30 [===================

Epoch 150/150
31/30 [==============================] - ETA: -2s - loss: 0.0659 - accuracy: 0.9826
Epoch 00150: saving model to training_1/cp.ckpt
31/30 [==============================] - 141s 5s/step - loss: 0.0659 - accuracy: 0.9826 - val_loss: 3.9955 - val_accuracy: 0.4061


In [17]:
test = os.listdir(MY_PATH+TEST_PATH)

In [18]:
test_df=raw(test,None)

In [19]:
test_df

,dirname,label
0,e86789078f3731bdc3d1e740825f29b3.jpg,None
1,6b0e58f46b51157485d532c8eeec6179.jpg,None
2,fd9cac41cda00325613b362025eb9cc1.jpg,None
3,10ede359e864dab7c381d9f8bb35de15.jpg,None
4,e73078a312effc5db81b849b083d1365.jpg,None
...,...,...
1995,5ba0a1dd4895ff62cd322dccbeab4d33.jpg,None
1996,39706bf6a1eb9fc681952d73cfdaa978.jpg,None
1997,09700570b778d58ab42a1aa8b62f4f83.jpg,None
1998,4cf03c70d04f9bcbf9918b896d21514f.jpg,None


In [20]:
# test datat function
def test_load_data(IMAGE_SIZE=256):
    print("Loading images...")
    test_data = []
    for i in range(len(test_df)):
        #print("Loading {0}".format(dirname))
        dirnames=test_df.iloc[i]["dirname"]
        img = Image.open(MY_PATH+TEST_PATH+dirnames)
        img = img.resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)
        #test_data.append([np.array(img),label])
        test_data.append(np.array(img))
    print("done")
    return test_data

test_data = test_load_data()

Loading images...
done


In [21]:
#load best weights
model.load_weights(checkpoint_path)

# pred output
test_images = np.array(test_data)
pred = model.predict(test_images)
pred_digits = np.argmax(pred,axis=1)
pred_digits

array([0, 3, 4, ..., 4, 1, 1])

In [22]:
test_df['label'] = pred_digits
test_df.rename(columns = {'dirname':'id','label':'flower_class'},inplace=True)

In [23]:
test_df['id'] = test_df['id'].apply(lambda x: x.replace('.jpg',''))
test_df.head()

,id,flower_class
0,e86789078f3731bdc3d1e740825f29b3,0
1,6b0e58f46b51157485d532c8eeec6179,3
2,fd9cac41cda00325613b362025eb9cc1,4
3,10ede359e864dab7c381d9f8bb35de15,1
4,e73078a312effc5db81b849b083d1365,4


In [24]:
test_df.to_csv('final.csv',index=False)

In [25]:
_, train_acc = model.evaluate(training_images, training_labels, verbose=0)
_, test_acc = model.evaluate(testing_images, testing_labels, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Train: 0.954, Test: 0.406


# Reference

In [ ]:
# mlp overfit on the moons dataset with patient early stopping and model checkpointing
from sklearn.datasets import make_moons
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model
# generate 2d classification dataset
X, y = make_moons(n_samples=100, noise=0.2, random_state=1)
# split into train and test
n_train = 30
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
# define model
model = Sequential()
model.add(Dense(500, input_dim=2, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=4000, verbose=0, callbacks=[es, mc])
# load the saved model
saved_model = load_model('best_model.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(trainX, trainy, verbose=0)
_, test_acc = saved_model.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))a